# ORM

In [2]:
#Istanziare l'app da terminale ed esplorare il model
from gnr.app.gnrapp import GnrApp
app = GnrApp('sandboxpg')
mydb = app.db
mymodel = mydb.model
model_structure = mymodel.src
model_structure['packages'].keys()

['sys',
 'adm',
 'glbl',
 'sandbox',
 'tutor',
 'test',
 'test15',
 'biz',
 'docu',
 'fatt',
 'email']

### La Table

In [3]:
#Definizione di una Table
class Table(object):
    def config_db(self, pkg):
        tbl = pkg.table('cliente', pkey='id',caption_field='ragione_sociale')
        tbl.column('id' ,size='22', name_long='Id')
        tbl.column('ragione_sociale', size=':40', name_long='Ragione sociale')
        tbl.column('indirizzo', name_long='Indirizzo')
        prov_col = tbl.column('provincia',size='2',name_long='Provincia')
        prov_col.relation('glbl.provincia.sigla', relation_name='clienti', mode='foreignkey')

In [4]:
model_structure['packages.fatt.tables.cliente.columns'].keys()

['id',
 '__ins_ts',
 '__del_ts',
 '__mod_ts',
 '__ins_user',
 'ragione_sociale',
 'indirizzo',
 'provincia',
 'comune_id',
 'cliente_tipo_codice',
 'pagamento_tipo_codice',
 'note',
 'email',
 'data_iscrizione_newsletter',
 'data_disiscrizione_newsletter']

### Le query

In [5]:
query_1 = mydb.table('fatt.cliente').query(columns='$ragione_sociale,@provincia.@regione.nome AS regione', limit=5)
query_1.fetch()

[[ragione_sociale=Camillozzi SPA,regione=Piemonte,pkey=huWxufwWNqeERsBSbHiXbw],
 [ragione_sociale=Bianchi Luigino,regione=Lombardia,pkey=pN1YrXqfPyinfCQBQ2qewQ],
 [ragione_sociale=Effelunga,regione=Lombardia,pkey=eXTjroRqOLqqaP5Ez4vL0g],
 [ragione_sociale=Rossi Mario,regione=Lombardia,pkey=mf_6EJbWN1unRWkVjRRTgA],
 [ragione_sociale=Giardini e Giardini,regione=Lombardia,pkey=ja14mgX2P2mRQWUNMWcdKg]]

In [6]:
query_2 = mydb.table('fatt.fattura').query(columns="""
                                                    $data,$protocollo,$totale_fattura,
                                                    @cliente_id.ragione_sociale AS cliente
                                                    """, 
                                                    where='$totale_fattura>1000',
                                                    limit=5, order_by='$data DESC')
for r in query_2.fetch():
    print ('Fattura {protocollo} del {data} Cliente: {cliente} Totale: {totale_fattura}'.format(**r))

Fattura F21/000010 del 2021-08-31 Cliente: Bianchi Luigino Totale: 1113.06
Fattura F16/000004 del 2016-09-09 Cliente: Galimberti srl Totale: 1626.1
Fattura F16/000003 del 2016-01-19 Cliente: Effelunga Totale: 1049.35
Fattura F15/000325 del 2015-04-14 Cliente: Rockerduck S.p.a. Totale: 46046.368
Fattura F15/000323 del 2015-04-14 Cliente: Galimberti srl Totale: 7118.58


In [7]:
print(query_2.sqltext)

SELECT  t0."data" AS "data",
t0."protocollo" AS "protocollo",
t0."totale_fattura" AS "totale_fattura",
t1."ragione_sociale" AS cliente,
t0.id AS pkey
 FROM fatt."fatt_fattura" AS t0
       LEFT JOIN fatt."fatt_cliente" AS t1 ON t1."id" = t0."cliente_id"
WHERE (t0."totale_fattura">1000) AND (t0."__del_ts" IS NULL)
ORDER BY t0."data" DESC
LIMIT 5


### Il metodo record

In [8]:
tbl = mydb.table('fatt.cliente')
cliente_id = 'mf_6EJbWN1unRWkVjRRTgA'
record_cliente = tbl.record(cliente_id, mode='bag')
print(record_cliente['@provincia.@regione.nome'])

Lombardia


In [9]:
for fatt in record_cliente['@fatture'].values():
    print (fatt['protocollo'],fatt['totale_lordo'])

F21/000013 None
F15/000135 191
F15/000241 184
F15/000258 1866
F15/000039 315.7
F15/000249 676
F15/000174 1180.8
F15/000194 837.6
F15/000129 1651.9
F15/000074 658.2
F15/000319 449
F15/000277 120
F15/000311 82
F15/000007 186
F15/000280 1642
F15/000308 365
F15/000130 806
F14/000002 71.4
F14/000012 61.2
F15/000217 1296
F15/000252 516
F15/000051 870
F15/000015 216
